In [10]:
from lmdb import Environment
import numpy as np
import os
import sys
import ipyvolume as ipv
import numpy as np
import ipyvolume.pylab as p3
from PythonProtocols.SceneSetup_pb2 import SceneSetup
from PythonProtocols.ScatterSample_pb2 import ScatterSample
from PythonProtocols.Result_pb2 import Result
from PythonProtocols.DisneyDescriptor_pb2 import DisneyDescriptor

BATCH_SIZE = 2048

class Dataset:
    
    def __init__(self):
        self.env = Environment("../Data/Dataset_WIP/Train.lmdb", map_size=int(1e9), subdir=False, max_dbs=128, mode=0, create=False, readonly=True)
        self.scatter_db = self.env.open_db(ScatterSample.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)
        self.scenes_db = self.env.open_db(SceneSetup.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)
        self.results_db = self.env.open_db(Result.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)
        self.descriptor_db = self.env.open_db(DisneyDescriptor.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)

In [11]:
from PythonProtocols.SceneSetup_pb2 import SceneSetup

dataset = Dataset()

clouds = {}
cloudNames = []

with dataset.env.begin() as transaction:
    cursor = transaction.cursor(dataset.scenes_db)
    cursor.first()
    for key, value in cursor:
        scene = SceneSetup()
        scene.ParseFromString(value)
        if scene.cloud_path not in clouds.keys():
            clouds[scene.cloud_path] = ([],[])
            cloudNames.append(scene.cloud_path)
        id = int.from_bytes(key, byteorder='little')
        clouds[scene.cloud_path][0].append(id)
        clouds[scene.cloud_path][1].append(scene)

In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from PythonProtocols.ScatterSample_pb2 import ScatterSample
from PythonProtocols.Result_pb2 import Result
from PythonProtocols.DisneyDescriptor_pb2 import DisneyDescriptor

cloud = widgets.Dropdown(
        options=cloudNames,
        value=cloudNames[0],
        description='Cloud:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

scene = widgets.Dropdown(
        options=clouds[cloudNames[0]][0] + [None],
        value=clouds[cloudNames[0]][0][0],
        description='Scene Id:',
        disabled=False
    )

def update_scene_options(*args):
    scene.options = clouds[cloud.value][0] + [None]
    scene.value = clouds[cloud.value][0][0]
    
cloud.observe(update_scene_options, 'value')

def normalized(v):
    magnitude = np.sqrt(np.dot(v, v))
    if magnitude == 0:
        magnitude = np.finfo(v.dtype).eps
    return v / magnitude

def update(cloud, sceneId):
    dataset = Dataset()
    
    with dataset.env.begin() as transaction:
        pointCount = BATCH_SIZE
        scenes = [sceneId]
        if sceneId is None:
            scenes = clouds[cloud][0]
            pointCount *= len(clouds[cloud][0])
        
        x = np.zeros(pointCount)
        y = np.zeros(pointCount)
        z = np.zeros(pointCount)

        u = np.zeros(pointCount)
        v = np.zeros(pointCount)
        w = np.zeros(pointCount)

        c = np.zeros((pointCount, 3))
        cSelected = np.zeros((pointCount, 3))
        
        sample = ScatterSample()
        radiance = Result()
        
        pointId = 0
        for sceneId in scenes:
            for i in range(BATCH_SIZE):
                sampleId = sceneId * BATCH_SIZE + i
                result = transaction.get(sampleId.to_bytes(4, 'little'), db=dataset.scatter_db)
                sample.ParseFromString(result)
                x[pointId] = sample.point.x
                y[pointId] = sample.point.y
                z[pointId] = sample.point.z

                u[pointId] = sample.view_direction.x
                v[pointId] = sample.view_direction.y
                w[pointId] = sample.view_direction.z

                result = transaction.get(sampleId.to_bytes(4, 'little'), db=dataset.results_db)
                if result is None:
                    c[pointId, :] = [0, 0, 1]
                else:
                    radiance.ParseFromString(result)
                    color = radiance.light_intensity / 4
                    c[pointId, :] = [color, color * 0.8, color * 0.6]
                
                pointId += 1
        fig = ipv.figure(width=700, height=700, lighting=False)
        q = ipv.quiver(x, y, z, u, v, w, color=c, size=2, color_selected='red', size_selected=2, selected=[-1])
        ipv.xyzlim(-0.5, 0.5)
        ipv.pylab.style.set_style_dark()
        #ipv.pylab.style.box_off()
        ipv.pylab.style.axes_off()
        p3.selector_default()
        fig.camera_control = 'orbit'
        
        def onSelection(_):
            if len(q.selected[0]) == 0:
                return
            x = []
            y = []
            z = []
            
            c = []
            
            selected = q.selected[0][0]
            
            sampleId = int(scenes[0] * BATCH_SIZE + selected)
            
            with dataset.env.begin() as transaction:
                sample = ScatterSample() 
                sample.ParseFromString(transaction.get(sampleId.to_bytes(4, 'little'), db=dataset.scatter_db))
                scene = SceneSetup()
                scene.ParseFromString(transaction.get(sceneId.to_bytes(4, 'little'), db=dataset.scenes_db))
                descriptor = DisneyDescriptor()
                descriptor.ParseFromString(transaction.get(sampleId.to_bytes(4, 'little'), db=dataset.descriptor_db))

                lightDirection = np.array([scene.light_direction.x, scene.light_direction.y, scene.light_direction.z])
                viewDirection = np.array([sample.view_direction.x, sample.view_direction.y, sample.view_direction.z])
                origin = np.array([sample.point.x, sample.point.y, sample.point.z])

                eZ = normalized(lightDirection);
                eX = normalized(np.cross(lightDirection, viewDirection));
                eY = np.cross(eX, eZ);
                
                sampleId = 0
                scale = 0.5 / (scene.cloud_size_m / 10)
                for layerId in range(10):
                    for zPos in range(-2, 7):
                        for yPos in range(-2, 3):
                            for xPos in range(-2, 3):
                                offset = (eX * xPos + eY * yPos + eZ * zPos) * scale;
                                pos = origin + offset;
                                
                                if (-0.5 < pos).all() and (pos < 0.5).all():
                                    x.append(pos[0])
                                    y.append(pos[1])
                                    z.append(pos[2])

                                    density = descriptor.grid[sampleId] / 128
                                    if density == 0:
                                        c.append([0, 0, 1])
                                    else:
                                        c.append([density, density, density])

                                sampleId += 1

                    scale *= 2;
                assert(sampleId == len(descriptor.grid))
            x = np.array(x)
            y = np.array(y)
            z = np.array(z)
            c = np.array(c)
            ipv.scatter(x, y, z, color=c, size=1, marker='sphere')
            q.size = 0.4
        fig.on_selection(onSelection)
        
        p3.show()

w = interactive(update, cloud=cloud, sceneId=scene)
display(w)

interactive(children=(Dropdown(description='Cloud:', layout=Layout(width='50%'), options=('10_FREEBIE_CLOUDS\\…